Steps experiments:
- remove news more than 5k words 
- use BPE https://github.com/google/sentencepiece compess datasets 
- split it(dev/valid/test)
- use dict from BPE to preprocess

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm_notebook
import os
import numpy as np
from datetime import datetime
pd.set_option('display.max_colwidth', -1)
import sentencepiece as spm


In [2]:
def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
def clean_news_text(text):
    return  re.sub("(&#(?:\d)*;)", "", re.sub("<.*?>", "", text)) \
    .replace("\n","").replace("\r","").replace("\x96","") \
    .replace("&nbsp;", "").replace("&mdash;", "")
    
create_dir('./data/opennmt/ria_bpe_1')
create_dir('./data/opennmt/ria_bpe_1/source')
create_dir('./data/opennmt/ria_bpe_1/bpe')

### Load  and Split long texts 

In [10]:
chunksize = 1000
news_reader = pd.read_json("./data/ria_news_dataset/ria.json.gz", lines=True,  dtype=['object', 'object'], compression='gzip', chunksize=chunksize)


In [11]:
limit=5000
count = 0
with(open("./data/opennmt/ria_bpe_1/bpe/ria_title_5000.txt", "w")) as f_title:
    with(open("./data/opennmt/ria_bpe_1/bpe/ria_text_5000.txt", "w")) as f_text:
        for news_chunk in tqdm_notebook(news_reader):
            news_chunk.text = news_chunk.text.apply(clean_news_text)
            for _, row in news_chunk.iterrows():
                if len(row.text) <=limit:
                    f_title.write(row.title +"\n")
                    f_text.write(row.text +"\n")
                    count +=1

print("All rows: ", count)

All rows:  958878


### BPE

In [ ]:
spm.SentencePieceTrainer.train('--input=./data/opennmt/ria_bpe_1/bpe/ria_title_5000.txt,./data/opennmt/ria_bpe_1/bpe/ria_text_5000.txt --model_prefix=./data/opennmt/ria_bpe_1/bpe1 --vocab_size=30000 --model_type=bpe')

In [ ]:
sp_bpe = spm.SentencePieceProcessor()
sp_bpe.load('./data/opennmt/ria_bpe_1/bpe1.model')